In [1]:
import pandas as pd
import numpy as np
import refinitiv.data as rd
import cufflinks as cf
from datetime import datetime
import calendar
from pandas.tseries.offsets import BMonthEnd
import warnings

cf.go_offline()
warnings.filterwarnings('ignore')

rd.open_session()

<refinitiv.data.session.Definition object at 0x7f429e2abc40 {name='codebook'}>

In [30]:
m_code = {1: 'F', 2: 'G', 3: 'H', 4: 'J', 5: 'K', 6: 'M',
          7: 'N', 8: 'Q', 9: 'U', 10: 'V', 11: 'X', 12: 'Z'}


def sys_year():
    today_date = datetime.strptime(
        datetime.today().strftime('%Y-%m-%d'), '%Y-%m-%d')
    y = str(today_date.year)[-2:]
    return y


def last_workday(d):
    last = BMonthEnd()
    return last.rollforward(d)


def eom(y, m):
    ed = last_workday(datetime(year=int(y), month=m, day=1)
                      ).strftime('%Y-%m-%d')
    return ed


def som(y, m):
    sd = datetime(year=int(y), month=m, day=1).strftime('%Y-%m-%d')
    return sd


def is_continuous(r):
    return 'c' in r


def gen_contracts(r, d, m):
    if m == None:
        now = datetime.now()
        m = now.month
    xd = rd.get_data(r, 'EXPIR_DATE')
    xd = xd['EXPIR_DATE'][0]
    _m = xd.month if m is None else m
    _y = xd.year
    _x = []
    x = []
    root = contract_data(r)
    if is_continuous(r):
        root += str(m_code[m])

    while d >= 0:
        if d == 0:
            _x = ('{}{}'.format(root, str(_y - d)[-1:]))
            _ed = eom(_y-d, _m)
        else:
            dec = str(_y - d)[-2:][0]
            _x = '{}{}^{}'.format(root, str(_y - d)[-1], dec)
            _ed = eom(_y-d, _m)
        _sd = som(_y-d, _m)
        x.append([_x, _sd, _ed])
        d -= 1
    return x


def contract_data(r):
    if is_continuous(r):
        root = r.split('c')[0]
    else:
        k = list(r)
        root = ''.join(k[:-1])
    return root


def get_timeseries_seq(l):
    d = {}
    for i in l:
        d[i[0]] = rd.get_history(
            i[0], 'SETTLE', start=i[1], end=i[2], interval='daily')
    return d


def get_data(ric, num_years=5, mm=None):

    frames = gen_contracts(ric, num_years, mm)
    hist = get_timeseries_seq(frames)
    seasonal = pd.DataFrame()
    for k in hist.keys():
        _idx = []
        for i in range(len(hist[k])):
            _idx.append(list(hist[k].index)[i].strftime('%b-%d'))
        hist[k]['Dates'] = _idx

    seasonal_df = pd.DataFrame(columns=['Dates'])
    for k in hist.keys():
        seasonal_df = pd.merge(seasonal_df, hist[k],on ='Dates', how='outer')
        seasonal_df.rename(columns = {'SETTLE':f'{k}'}, inplace = True)
    seasonal_df = seasonal_df.sort_values(by='Dates')
    seasonal_df = seasonal_df.fillna(method='backfill')
    seasonal_df = seasonal_df.reset_index(drop=True)
    seasonal_df.index = seasonal_df['Dates']
    seasonal_df = seasonal_df.drop(columns='Dates')
    seasonal_df = seasonal_df.astype('float')
    seasonal_df.iplot(theme='solar', colors=['#6978F7', '#A325E9', '#96E05D', '#4A7FB9', '#E75A2D', '#FBE55A', '#8C8C8D', '#5A54F6'])

    return seasonal_df


In [ ]:
rd.close_session()